In [1]:
import os
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS\\research'

In [2]:
os.chdir("../")
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreparationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: str

In [4]:
from docClassify.constants import *
from docClassify.utils.common import read_yaml, create_directories, create_bounding_box

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_preparation_config(self) -> DataPreparationConfig:
        config = self.config.data_preparation

        create_directories([config.root_dir])

        data_preparation_config = DataPreparationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_dir=config.unzip_dir
        )

        return data_preparation_config

In [6]:
import os
from docClassify.logger import logger

In [9]:
import easyocr
from tqdm.notebook import tqdm
import json

class DataPreparation():
    def __init__(self, config:DataPreparationConfig):
        self.config = config
        #print(Path(self.config.root_dir).glob("*/*.png"))
        self.image_paths = sorted(list(Path(self.config.unzip_dir).glob("*/*.png")))
        #print(Path(self.config.root_dir).glob("*/*.png"))

    def prepare_all_files(self) -> bool :
        preparation_status = False
        try:
            reader = easyocr.Reader(['en'])
            for image_path in tqdm(self.image_paths):
                print(image_path)
                ocr_result = reader.readtext(str(image_path))
            
                ocr_page = []
                for bbox, word, confidence in ocr_result:
                    ocr_page.append({
                        "word": word, "bounding_box": create_bounding_box(bbox)
                    })
                
                with image_path.with_suffix(".json").open("w") as f:
                    json.dump(ocr_page, f)
                    
            preparation_status = True
            return preparation_status
            
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_preparation_config = config.get_data_preparation_config()
    data_preparation = DataPreparation(config=data_preparation_config)
    data_preparation.prepare_all_files()
except Exception as e:
    raise e

[2024-02-05 23:38:04,912: INFO: common: yaml file: src\docClassify\config\config.yaml loaded successfully]
[2024-02-05 23:38:04,918: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 23:38:04,923: INFO: common: created directory at: artifacts]
[2024-02-05 23:38:04,927: INFO: common: created directory at: artifacts/data_preparation]
[2024-02-05 23:38:04,932: WARNING: easyocr: Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.]


  0%|          | 0/50 [00:00<?, ?it/s]

artifacts\data_ingestion\data\balance sheet\bs1.png
artifacts\data_ingestion\data\balance sheet\bs10.png
artifacts\data_ingestion\data\balance sheet\bs11.png
artifacts\data_ingestion\data\balance sheet\bs12.png
artifacts\data_ingestion\data\balance sheet\bs13.png
artifacts\data_ingestion\data\balance sheet\bs14.png
artifacts\data_ingestion\data\balance sheet\bs15.png
artifacts\data_ingestion\data\balance sheet\bs16.png
artifacts\data_ingestion\data\balance sheet\bs17.png
artifacts\data_ingestion\data\balance sheet\bs2.png
artifacts\data_ingestion\data\balance sheet\bs3.png
artifacts\data_ingestion\data\balance sheet\bs4.png
artifacts\data_ingestion\data\balance sheet\bs5.png
artifacts\data_ingestion\data\balance sheet\bs6.png
artifacts\data_ingestion\data\balance sheet\bs7.png
artifacts\data_ingestion\data\balance sheet\bs8.png
artifacts\data_ingestion\data\balance sheet\bs9.png
artifacts\data_ingestion\data\cashflow\cf1.png
artifacts\data_ingestion\data\cashflow\cf10.png
artifacts\dat